Import required libraries

In [1]:
import ast
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import warnings
pd.set_option('display.max_colwidth', None)
warnings.filterwarnings("ignore")

Load the data from CSV

In [2]:
df = pd.read_csv('./data/goodnotes_submission.csv').reset_index(drop=True)

In [3]:
df.head()

submission_id  \
0        aglcrj   
1        agoowm   
2        agprta   
3        agpzxb   
4        agq8qv   

                                                                                        submission_title  \
0                                                                  Goodnotes 4 vs. Goodnotes 5 right now   
1                                                                              The bundle is available !   
2  Finally... GoodNotes 5! Did I miss anything in this video? Tried to point out all the "new" features.   
3                                                                What happened to the pen (Goodnotes 5)?   
4                                                              Non Apple Pencil styluses on GOodnotes 5?   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     submission_selftext  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    I have used Goodnotes 4 for work a ton.  And I do mean a ton.  I bought and downloaded 5 yesterday, transfer was easy.  I realized I can't use the Mac App with 5.  Right?  Changes I made in 5 aren't synced to the desktop app.  Also I kept getting a syncing error in 5.  What was it syncing with?  I am going to continue to play with it but I am not sure I feel comfortable diving in yet.   
1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    NaN   
2                                                                                                                                                                                 

Convert `reply_body` from a string to a list of strings

In [4]:
df['reply_body'] = df['reply_body'].apply(ast.literal_eval)

Combine `submission_title`, `submission_selftext`, and first three entries of `reply_body` into one column

In [5]:
df['all_text'] = df['submission_title'] + ' ' + df['submission_selftext'].apply(lambda selftext: selftext if selftext is not np.nan else '') + ' ' + df['reply_body'].apply(lambda replies: ' '.join([reply for reply in replies[:3]]) if replies is not np.nan else '')

In [6]:
df.head()

submission_id  \
0        aglcrj   
1        agoowm   
2        agprta   
3        agpzxb   
4        agq8qv   

                                                                                        submission_title  \
0                                                                  Goodnotes 4 vs. Goodnotes 5 right now   
1                                                                              The bundle is available !   
2  Finally... GoodNotes 5! Did I miss anything in this video? Tried to point out all the "new" features.   
3                                                                What happened to the pen (Goodnotes 5)?   
4                                                              Non Apple Pencil styluses on GOodnotes 5?   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     submission_selftext  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    I have used Goodnotes 4 for work a ton.  And I do mean a ton.  I bought and downloaded 5 yesterday, transfer was easy.  I realized I can't use the Mac App with 5.  Right?  Changes I made in 5 aren't synced to the desktop app.  Also I kept getting a syncing error in 5.  What was it syncing with?  I am going to continue to play with it but I am not sure I feel comfortable diving in yet.   
1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    NaN   
2                                                                                                                                                                                 

Split dataframe into two, depending on whether `submission_link_flair_text` is present or not

In [7]:
unlabelled_df = df[pd.isnull(df['submission_link_flair_text'])]
labelled_df = df[~pd.isnull(df['submission_link_flair_text'])]

In [8]:
unlabelled_df['submission_link_flair_text'].value_counts(dropna=False)

NaN    438
Name: submission_link_flair_text, dtype: int64

In [9]:
labelled_df['submission_link_flair_text'].value_counts(dropna=False)

Question - iPad      140
Templates             39
Question - Other      30
Review                17
Question - Mac         8
Stylus problems        5
Question - iPhone      2
Name: submission_link_flair_text, dtype: int64

Combine `Question - x` in `submission_link_flair_text` into one category called `Question`

In [10]:
labelled_df['submission_link_flair_text'] = labelled_df['submission_link_flair_text'].str.replace(pat='Question - \\w+', repl='Question', regex=True)

In [11]:
labelled_df['submission_link_flair_text'].value_counts(dropna=False)

Question           180
Templates           39
Review              17
Stylus problems      5
Name: submission_link_flair_text, dtype: int64

Add a `prefix` column and rename some existing columns in `labelled_df`

In [12]:
labelled_df['prefix'] = 'tag classification'
labelled_df = labelled_df.rename(columns={'all_text': 'input_text', 'submission_link_flair_text': 'target_text'})
labelled_df = labelled_df[['prefix', 'input_text', 'target_text']]

In [13]:
labelled_df.head()

,prefix,input_text,target_text
2,tag classification,"Finally... GoodNotes 5! Did I miss anything in this video? Tried to point out all the ""new"" features. Man i imagined you would be a lot more popular in this sub. Love your channel :) Thank you :)",Review
22,tag classification,"I haven't had GN4 and I‘m wondering if I should go with the GN5 bundle or just buy GN5 as is. Hey there! I've been hearing a lot about GoodNotes and since I've finally upgraded to an iPad Pro where my Pencil works well, I was wondering if it's better for me to get the GN5 bundle (the one where I pay for GN4 and download GN5 for free) or go straight to paying for GN5?\n\nI saw how buggy GN5 is, seeing how it's still in beta, and thought of opting out to GN4 in the meantime until the bugs are fixed. Probably a better option to go with Notability seeing as the issues still haven't been fixed months after the release... GN4 has its own issues that may or may not effect you. I’d just hard pass on the lot and look elsewhere. \n\nThere’s no point getting used to the workflow of a deprecated piece of software (GN4) hoping that GN5 will become usable at some point. \n\nGN5 not in beta though, it’s released and people have paid for it. \n\nNotability, Note Shelf 2 are two reasonable alternatives to investigate. There are others but those are the ones I’d recommend. I’m a long time GN4 user and didn’t have any problems with it. GN5 has its problems, probably released too early, but they are working hard to address them. \nGN beats Notability in the way notes are organised and the ease of creating new papers - I like notability for its ink and pens, but the key to note taking is being able to find what you previously wrote. \nMy advice, and only mine, would be to go with GN4 and give them a month or so to resolve the bugs, they are close now",Question
27,tag classification,"Colors Is there a way to find the color used? Meaning I write something in a custom pink and then change colors but then want to use that color again. Thanks Can be done if your have an app like Artstudio that has a colorselector tool, that way you can take a screenshot &amp; find the hex code for the color. I do this in Procreate all the time. I have a stack of color swatches and a saved pallet of all the colors I save ! Pinterest has a ton of color palettes too which I love 😄 Can you link some cool color pallets here?",Question
30,tag classification,"How do I make detailed outlines in GN5? I know how to add pages to outlines, but I’m wondering how to put pages into categories in the outline, like some pdfs do.\n\nFor instance, within chapter 1, there may be pages 1, 42, and 69. Can I make these detailed outlines for my own notebooks as well?\n\nThanks! I hope you get a reply, I’m wondering the same. Also have some items in an outline that I cannot edit or delete.",Question
34,tag classification,"Writing to text conversion? Wondering if there is any other method of converting written text to typed text than the copy/paste? That is rather annoying. He’s right though, mando content is a sequel meme. You start with 2 and once you use those two you can only get it back with primary kills, so if you wanna ape someone you need to get a primary kill first Ekchoti yo post ni hernu: https://www.reddit.com/r/technepal/comments/ns59a2/mobile_ra_sim_ma_imei_imsi_ra_iccid_vaneko_k_ho/?utm_medium=android_app&amp;utm_source=share",Question


Split `labelled_df` into train and test sets

In [14]:
train_tag_df, val_tag_df = train_test_split(labelled_df, stratify=labelled_df['target_text'], test_size=0.2, random_state=42)

In [15]:
train_tag_df.shape

(192, 3)

In [16]:
val_tag_df.shape

(49, 3)

Save train, test, and unlabelled data as CSVs

In [17]:
train_tag_df.to_csv('./data/singletask_noupsampling_train.csv', index=False)
val_tag_df.to_csv('./data/singletask_noupsampling_val.csv', index=False)
unlabelled_df.to_csv('./data/unlabelled_df.csv', index=False)

See how many samples of each label/tag are in the train set

In [18]:
train_tag_df['target_text'].value_counts(dropna=False)

Question           143
Templates           31
Review              14
Stylus problems      4
Name: target_text, dtype: int64

Write a function to upsample data based on given label/tag

In [19]:
def upsample_train(df, col_name, label, freq):
    df_sample = df[df[col_name] == label]
    for i in range(freq):
        df = pd.concat([df, df_sample]).reset_index(drop=True)
    return df

Upsample data with the labels `Review` and `Stylus problems` three times and those with the label `Templates` two times

In [20]:
train_tag_df = upsample_train(train_tag_df.copy(), 'target_text', 'Review', 4)
train_tag_df = upsample_train(train_tag_df.copy(), 'target_text', 'Stylus problems', 4)
train_tag_df = upsample_train(train_tag_df.copy(), 'target_text', 'Templates', 2)

In [21]:
train_tag_df['target_text'].value_counts()

Question           143
Templates           93
Review              70
Stylus problems     20
Name: target_text, dtype: int64

Save upsampled train data as CSV

In [22]:
train_tag_df.to_csv('./data/singletask_train.csv', index=False)